<a href="https://colab.research.google.com/github/bapatmugdha/DataScienceSummit20/blob/master/GeneticAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import necessary libraries

In [1]:
import random  as random
import numpy as np


## Create Population Class

In [2]:
class Population:
#Initialize the Population , random  chessboard states
## GA setup
    def __init__(self,num_queens=8,population_size=50, mutationRate=0.4):
        self.num_queens = num_queens
        self.population_size= population_size
        self.population = self.makePopulation()
        self.matingpool = []
        self.populationfitness=[]
        self.mutationRate = mutationRate
        self.populationfitness = self.calcFitness()

##generate Population of given size 
    def makePopulation(self):
        population = []
        boardsize = self.num_queens
        for i in range(self.population_size):
            population.append(Board(boardsize))
        return population
    
##Calculate fitness for the entire population   
    def calcFitness(self):
        fitness= []
        pop= self.population
        for cboards in pop:
            fitness.append(cboards.getFitness())
        return fitness 
    
#generate a mating pool    
    def naturalSelection(self):
        self.matingpool = []
        maxval = max(self.populationfitness)
        self.matingpool = self.population
        for i in range(self.population_size):
            probconvert = np.floor(((((maxval-self.populationfitness[i])/maxval)*(1-0))+0)*100)
            for index in range(int(probconvert)):
                self.matingpool.append(self.population[i])
      

        
## generate new population    
    def generate(self):
        #print("poplength",len(self.population))
        for i in range(self.population_size):
            PartnerAindex = random.randint(0,self.population_size-1)
            PartnerBindex = random.randint(0,self.population_size-1)
            #print(PartnerAindex,PartnerBindex)
            PartnerA = self.matingpool[PartnerAindex] 
            PartnerB = self.matingpool[PartnerBindex] 

            child = PartnerA.crossover(PartnerB)
            child.mutate(self.mutationRate)
            self.population[i] = child
            
            

        
## GA algorithm order    
        
        ##natural selection
        ## update population
        ## recalculate fitness
    def draw(self):
         self.naturalSelection()
         self.generate()
         self.populationfitness =[]
         #print(self.populationfitness)
         self.populationfitness = self.calcFitness()
         #print(self.populationfitness)



## Create Board Classs

In [3]:
class Board:
## initialize board with random positions for the Queen
    def __init__(self,size):
        self.fitness = 0
        self.boardsize = size
        self.board = self.create_Board()

##use 1D to represnt the board
## indexes indicate rows and values indicate columns
## create board such that queens are not placed in same column        
    def create_Board(self):
        board = [ x  for x in range(self.boardsize)]
        random.shuffle(board)
        return board
    
##Find the number of Queens that attack the given queen    
    def getFitness(self):
        n = len(self.board)
        diagNWSE = {}
        diagSWNE = {}
        cnt =0
        row= []
        col= []
        cost =0
  
# reset attack on cells to  0
# 0 indicates no attack
# n indicates attack by n queens 
        for i in range(n):
            row.append(0)
            col.append(0)
            
        for cnt in range(-(n-1),n):
            diagNWSE[cnt]= 0
        for cnt in range(2*n-1):
            diagSWNE[cnt]= 0

# calculate attack on each by all queens  
        for i in range(n):
            j= self.board[i]
            diagNWSE[j-i] +=1 
            diagSWNE[j+i] +=1
            row[i]+=1
            col[j]+=1
# calculate the attack on each queen for the given board configuration           
        costboard = []
        for i in range(n):
            j= self.board[i]
            costboard.append(diagNWSE[j-i]- 1 + diagSWNE[j+i]-1+row[i]-1+col[j]-1)
            cost= max(costboard)
        return cost
        
#define the crossover process.
# accept the values on positions where they match for both partners
# select the remaining values from possible values for remaining locations        
    def crossover(self,partner):
        child = Board(self.boardsize)
        n = len(self.board)
        current = [ -1 for x in range(n)]
        possible = [ x  for x in range(n)]

        
        for i in range(len(child.board)):
                if self.board[i]== partner.board[i]:
                    child.board[i]=self.board[i]
                    possible.remove(self.board[i])
                    current[i]=self.board[i]

                    
        for i in range(len(child.board)):
                if current[i] == -1:
                    current[i] = random.choice(possible)
                    child.board[i] = current[i]
                    possible.remove(current[i])
           
        return child
        

# mutation  by random swapping of column values    
    def mutate(self,mutationRate):
            randno = random.random()
            n = len(self.board)
            boardnew = self.board
            if randno< mutationRate:
                pos1 = random.randint(0,n-1)
                pos2 = random.randint(0,n-1)

                boardnew[pos1], boardnew[pos2] = boardnew[pos2], boardnew[pos1]
                self.board = boardnew    
   


# Main Function

In [4]:
def main():
    # main
    #setup         
    mutationRate = 0.01
    totalPopulation = 150                    
    b= Population(population_size=totalPopulation, mutationRate=mutationRate)               
    
    ## draw
    generations = 0
    minval = min(b.populationfitness)

    while minval!=0:
        b.draw()
        minval = min(b.populationfitness)
        generations += 1
    else:
        idx = b.populationfitness.index(0)
        print("Number of generations to reach the solution",generations)
        possoln= b.population[idx].board
        print("Possible N Queen Solution",b.population[idx].board) 
        print("Cost of the solution",b.populationfitness[idx])
        for i in range(len(possoln)):
            leftj= possoln[i]
            rightj = 7-leftj
            cnt =0
            print("====="*8,end="=\n")
            for cnt in range(leftj):
                print("|    ",end='')
            print("| Q  ",end='')
            for cnt in range(rightj+1):
                print("|    ",end='')
            print("\n")
        print("====="*8,end="=\n")

main()    


Number of generations to reach the solution 2
Possible N Queen Solution [4, 6, 1, 5, 2, 0, 3, 7]
Cost of the solution 0
|    |    |    |    | Q  |    |    |    |    

|    |    |    |    |    |    | Q  |    |    

|    | Q  |    |    |    |    |    |    |    

|    |    |    |    |    | Q  |    |    |    

|    |    | Q  |    |    |    |    |    |    

| Q  |    |    |    |    |    |    |    |    

|    |    |    | Q  |    |    |    |    |    

|    |    |    |    |    |    |    | Q  |    

